In [169]:
import es_core_news_sm
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import regex
import pandas as pd
from scipy.spatial.distance import cosine
import os
import nltk

In [148]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/parker/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to

True

In [149]:
def EntrenarModelo(oraciones,NombreModelo):
    model = Word2Vec(oraciones,vector_size=1250, window=2, min_count=1)
    model.save(NombreModelo)
    
def CargarModelo(NombreModelo):
   modelo = Word2Vec.load(NombreModelo)
   vocabulario = [term for term in modelo.wv.key_to_index]  
   return(modelo,vocabulario)

In [150]:
def ObtenerEmbeddingOracion(modelo, oracion):
   Lista_vectores = []
   for w in nltk.sent_tokenize(oracion, language="spanish"):
       # Verificar que la palabra w exista en el modelo
       try:
           modelo.wv[w]
       except KeyError:
           continue
       # Obtener vector de la palabra
       vec = modelo.wv[w]
       Lista_vectores.append(vec)
   embedding_palabras = np.array(Lista_vectores)
   if (len(embedding_palabras) > 0):
        embedding_oracion = embedding_palabras.mean(axis=0)
   else:
        embedding_oracion = np.zeros(modelo.vector_size)
   return(embedding_oracion) 

In [151]:
def CrearCorpus(path):
  directorio = os.listdir(path)
  corpus = []
  doc_id = []  
  for filename  in directorio:
     texto = open(path+filename,'r',encoding="UTF-8").read()
     corpus.append(texto)
     doc_id.append(filename)
  return(corpus,doc_id)

def PreProcesarOraciones(textos):
    texto_limpio = []
    for texto in textos:
        if len(texto) != 0:
            texto_limpio.append(nltk.sent_tokenize(texto, language="spanish"))
    return texto_limpio

In [152]:
def CrearDiccionario(lista,claves):
   dicc = {}
   for  v in range(0,len(claves)):
      dicc[claves[v]] = lista[v]
   return(dicc)

In [153]:
PATH = "DiscursosOriginales/"
nlp          = es_core_news_sm.load()
corpus,docID = CrearCorpus(PATH)
oraciones = PreProcesarOraciones(corpus)
CorpusConClave  = CrearDiccionario(corpus,docID)


In [154]:
EntrenarModelo(oraciones,'mi_word2vec')

In [155]:
modelo, vocabulario = CargarModelo('mi_word2vec')

In [156]:
doc1 = CorpusConClave['73117.txt']
doc2 = CorpusConClave['136426.txt']
vec1 = ObtenerEmbeddingOracion(modelo, doc1)
vec2 = ObtenerEmbeddingOracion(modelo, doc2)

similitud = 1-cosine(vec2, vec1)
print(similitud)

0.018361229449510574


In [241]:
# Ordenar oraciones por puntaje
def centroide(modelo, oraciones):
    embedding = []
    for oracion in oraciones:
        embedding.append(ObtenerEmbeddingOracion(modelo, oracion))
    return np.mean(embedding, axis=0)
def OrdenarOraciones(oraciones):
    puntaje = []
    centroidee = centroide(modelo, oraciones)
    for oracion in oraciones:
        puntaje.append(cosine(ObtenerEmbeddingOracion(modelo, oracion),centroidee))
    oraciones.sort(key=lambda oracion: cosine(ObtenerEmbeddingOracion(modelo, oracion),centroidee), reverse=True)
    return oraciones

In [ ]:
def generar_resumen(O,N,P,U):
    O = OrdenarOraciones(O)
    largo = 1
    Resumen = []
    M = len(O)
    for i in range(M):
        if (largo > N): return Resumen
        Vo = ObtenerEmbeddingOracion(modelo, O[i])
        incluirOracion = False
        for j in range(largo):
            Vr = ObtenerEmbeddingOracion(modelo, Resumen[j])
            Sim = 1-cosine(Vo, Vr)
            if Sim > U and (O[i] not in Resumen):
                incluirOracion = True
        if incluirOracion == True:
            Resumen[largo] = O[i]
            largo+=1